# Usar `transform` sobre `GroupedDataFrame`s


El método `transform` permite calcular, para cada grupo, una o varias columnas con el mismo `index` que el grupo, por lo tanto con el mismo número de filas y las mismas etiquetas. Por ejemplo puedo, dentro de cada grupo, normalizar los valores respecto a la media y la desviación típica del grupo.

Preliminares

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIRECTORY = Path('..') / 'data'

Consideramos el DataFrame:

In [3]:
df = pd.DataFrame(
    {
        "X": ['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c'], 
        "Y": np.arange(8),
        "Z": np.arange(8,16)
    }
)
df

,X,Y,Z
0,a,0,8
1,a,1,9
2,a,2,10
3,a,3,11
4,b,4,12
5,b,5,13
6,c,6,14
7,c,7,15


Agrupamos según los valores de `X`. En el vídeo anterior vimos cómo aplicar el método `agg`, pasándole la función para calcular el indicador.

In [5]:
df_agrupado = df.groupby("X")

In [6]:
df_agrupado.mean()

,Y,Z
X,,
a,1.5,9.5
b,4.5,12.5
c,6.5,14.5


Hagamos lo mismo pero usando `transform`

In [7]:
df_agrupado.transform(np.mean)

,Y,Z
0,1.5,9.5
1,1.5,9.5
2,1.5,9.5
3,1.5,9.5
4,4.5,12.5
5,4.5,12.5
6,6.5,14.5
7,6.5,14.5


In [9]:
df[["Y_media","Z_media"]] = df_agrupado.transform(np.mean)
df

,X,Y,Z,Y_media,Z_media
0,a,0,8,1.5,9.5
1,a,1,9,1.5,9.5
2,a,2,10,1.5,9.5
3,a,3,11,1.5,9.5
4,b,4,12,4.5,12.5
5,b,5,13,4.5,12.5
6,c,6,14,6.5,14.5
7,c,7,15,6.5,14.5


Se puede añadir las columnas proporcionadas por `transform` a nuestro DataFrame inicial

In [14]:
# Creamos una copia del DataFrame original


Añadimos las columnas que contienen las medias de Y y Z desglosadas por grupo

# Ejemplo de uso: normalización de columnas
Vamos ahora a llevar a cabo la normalización de los valores de Y y Z, teniendo en cuenta los grupos formados por los valores de X.
Si las columnas tienen ordenes de magnitud diferentes, para evitar que una columna domine totalmente el análisis, antes de aplicar un algoritmo de aprendizaje máquina, es recomendable normalizar sus valores. Normalmente se resta a cada columna su media y se divide por su desviación típica.

In [15]:
# Creamos una copia del DataFrame original
df_copia = df.copy()
df_copia.drop(columns=["Y_media","Z_media"], inplace=True)

In [16]:
df_copia.groupby("X").transform(lambda x: (x -x.mean()) / x.std())

,Y,Z
0,-1.161895,-1.161895
1,-0.387298,-0.387298
2,0.387298,0.387298
3,1.161895,1.161895
4,-0.707107,-0.707107
5,0.707107,0.707107
6,-0.707107,-0.707107
7,0.707107,0.707107


Podemos sustituir las columnas Y y Z por sus equivalentes normalizados, por grupo.

# Sustitución de valores faltantes por la media de su grupo

Modificamos `df` para introducir valores faltantes en sus columnas:

In [ ]:
df.loc[[0, 2, 5], 'Y'] = np.NaN
df.loc[6, 'Z'] = np.NaN
df

Vamos a pasar a `transform` una función que tomando una columna, recorra sus elementos, si el elemento no falta, lo deja intacto. Si el elemento falta, lo sustituye por la media de la columna del grupo al que pertenece.

Para ello, vamos a usar el método `where`, ver [referencia](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html)

> `where` admite como parámetro un vector booleano y un vector `other`. Si el elemento del vector booleano es `True`, deja intacto el elemento de la columna a la que se aplica. Si es `False`, usa el elemento correspondiente del vector `other`.

Ejemplo: aplicamos el método `where` a la columna Y, para sustituir los valores faltantes por el valor 1000.

In [ ]:
# Usamos ~ para negar un vector booleano, True se vuelve False y viceversa


## Ahora podemos sustituir los valores faltantes de Y y Z por la media de su grupo

In [17]:
notas = pd.read_csv(DATA_DIRECTORY / "notas.csv")
notas

,expediente,asignatura,ects,nota
0,2341,Inglés,4.5,7.0
1,2341,IDS,6.0,9.5
2,2341,TFG,12.0,9.0
3,608,Inglés,4.5,8.0
4,608,IDS,6.0,7.0
5,608,TFG,12.0,8.0
6,37,Inglés,4.5,6.0
7,37,IDS,6.0,10.0
8,37,TFG,12.0,10.0


In [18]:
notas_agrupadas = notas.groupby("expediente")

In [19]:
for expediente, grupo in notas_agrupadas:
    print(f"Expediente:{expediente}")
    print(f"{grupo}")

Expediente:37
   expediente asignatura  ects  nota
6          37     Inglés   4.5   6.0
7          37        IDS   6.0  10.0
8          37        TFG  12.0  10.0
Expediente:608
   expediente asignatura  ects  nota
3         608     Inglés   4.5   8.0
4         608        IDS   6.0   7.0
5         608        TFG  12.0   8.0
Expediente:2341
   expediente asignatura  ects  nota
0        2341     Inglés   4.5   7.0
1        2341        IDS   6.0   9.5
2        2341        TFG  12.0   9.0


In [20]:
def calcular_media_ponderada(notas_exp):
    media_ponderada = (notas_exp["nota"] * notas["ects"]).sum() / notas_exp["ects"].sum()
    return media_ponderada

In [21]:
expediente_37 = notas_agrupadas.get_group(37)
expediente_37

,expediente,asignatura,ects,nota
6,37,Inglés,4.5,6.0
7,37,IDS,6.0,10.0
8,37,TFG,12.0,10.0


In [22]:
calcular_media_ponderada(expediente_37)

9.2

In [23]:
notas_agrupadas.apply(calcular_media_ponderada)

expediente
37      9.200000
608     7.733333
2341    8.733333
dtype: float64